In [177]:
import gensim
import multiprocessing
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer
import numpy as np
import os
from operator import getitem
import pandas as pd
import re
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [178]:
stopwords_set = set(stopwords.words('english'))
required = {"needn", "doesn", "won", "shan't", "don", "ain", "not", "didn", "hadn", "haven't", "couldn't", "wasn't", "aren't", "isn", "needn't", "aren", "wouldn", "shouldn", "hasn't", "shan", "no", "wasn", "nor", "hasn", "mightn", "doesn't", "against", "wouldn't", "couldn", "hadn't", "isn't", "mustn", "don't", "weren't", "haven", "mustn't", "shouldn't", "weren", "won't", "mightn't"}
stopwords_set -= required
print(stopwords_set)
dictionary = set(nltk.corpus.words.words())

{'yours', 'should', 'while', 'such', 'doing', 'yourself', 'can', 'that', 'she', 'was', 'than', 'its', 'during', 'there', 'them', 't', 'ma', 'his', 'a', 'these', 'herself', 'other', 'again', 'about', 'which', 'each', 'once', "you'll", 'is', 'our', 'and', 'under', 'ours', 'most', 'some', 'whom', 'themselves', 'why', 'were', 'it', 'you', 'when', 'out', 'own', 'what', 'to', 'too', 'same', 'those', 'of', 'my', 'few', 'further', 'himself', 'hers', 'having', 'any', 'just', 'do', 'him', 'so', 'over', 'where', 'be', 'both', "that'll", 'above', 'had', "it's", 'did', 'will', 'being', 'before', 'your', 'm', 'y', 'only', 'now', 'off', 'an', 'from', 's', 'has', 'here', 'for', 'with', 'down', 'in', 'more', 'because', 'have', 'if', 'ourselves', 'all', 'the', "you'd", 'after', 'between', 'd', 'll', 'we', 'are', 'but', 'i', 'who', 'o', 'until', 'me', 'theirs', 'through', 'below', 've', 'this', 'myself', 'yourselves', 'how', 'he', 'am', "should've", "you've", "didn't", 'into', 'her', "she's", 'their', 'a

In [179]:
df = pd.read_csv("FinalTweetList_train.csv", header = None)
df[1].value_counts()

displeasure      5506
miscellaneous    4493
compliment       1380
Name: 1, dtype: int64

In [180]:
#downsample displeasure and miscellaneous to equalise classes


compliment = df[df[1]=='compliment']
displeasure = df[df[1]=='displeasure']
miscellaneous = df[df[1]=='miscellaneous']
_, displeasure = train_test_split(displeasure, test_size = len(compliment), random_state = 21)
_, miscellaneous = train_test_split(miscellaneous, test_size = len(compliment), random_state = 21)
df = compliment.append([displeasure, miscellaneous])
df = df.reset_index(drop = True)
print(df)

#stratified downsample

                                                      0              1
0     @TheOfficialSBI  @Indiapnb @IDBI_Bank @AxisBan...     compliment
1     First hand experience of the #CashRush process...     compliment
2     @ksmkkbookscom @HDFC_Bank yeah @AxisBank @Axis...     compliment
3     @narendramodi much of the issues will be sorte...     compliment
4     @AxisBankSupport Thank u for ur response right...     compliment
...                                                 ...            ...
4135  #Bankdeposits will spike: How Rs 500, Rs 1000 ...  miscellaneous
4136  @ICICIBank_Care got a call from +919821294665 ...  miscellaneous
4137  @keshri_niraj @AxisBankSupport Hi, could you p...  miscellaneous
4138  Nw banker will undrstnd wht a softwaredevolope...  miscellaneous
4139  @TheOfficialSBI @SBICard_Connect I have blocke...  miscellaneous

[4140 rows x 2 columns]


In [181]:
def processTweet(tweet):
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+','AT_USER',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #trim
    tweet = tweet.strip('\'"')
    #Convert to lower case
    tweet = tweet.lower()
    return tweet

for i in range(len(df[0])):
    df[0][i] = processTweet(df[0][i])

print(df)

                                                      0              1
0     at_user at_user at_user at_user at_user at_use...     compliment
1     first hand experience of the cashrush process ...     compliment
2     at_user at_user yeah at_user at_user r the mos...     compliment
3     at_user much of the issues will be sorted if b...     compliment
4     at_user thank u for ur response right now. thi...     compliment
...                                                 ...            ...
4135  bankdeposits will spike: how rs 500, rs 1000 n...  miscellaneous
4136  at_user got a call from +919821294665 some ici...  miscellaneous
4137  at_user at_user hi, could you please elaborate...  miscellaneous
4138  nw banker will undrstnd wht a softwaredevolope...  miscellaneous
4139  at_user at_user i have blocked my debit card f...  miscellaneous

[4140 rows x 2 columns]


In [182]:
train = df

def word_clean(words):
    #return ' '.join([word for word in words if (word.isalpha())])
    #return ' '.join([word for word in words if (word.isalpha() and word in dictionary)])
    return ' '.join([word for word in words if (word.isalpha() and word not in stopwords_set)])
    #return ' '.join([word for word in words if (word.isalpha() and word not in stopwords_set and word in dictionary)])
    
def clean_tweets(dataframe):
    return [word_clean(WordPunctTokenizer().tokenize(dataframe[0][i])) for i in range(len(dataframe[0]))]

x_train = clean_tweets(train)
y_train = train[1]
print(len(y_train))

4140


In [183]:
x_train = [gensim.utils.simple_preprocess(text) for text in x_train]
#Continous bag of words used
w2v_model = gensim.models.Word2Vec(min_count=1, window=5,
                                        size=100,
                                        workers=multiprocessing.cpu_count())
w2v_model.build_vocab(x_train)
w2v_model.train(x_train, total_examples=w2v_model.corpus_count, epochs=100)
print("Word2Vec trained")

x_train = [' '.join(i) for i in x_train]

Word2Vec trained


In [184]:
def word_feats(words):
    return {tuple(w2v_model.wv[word].tolist()): True for word in words}
    
def extract_feats(xdataframe, ydataframe):
    #return [(word_feats(WordPunctTokenizer().tokenize(xdataframe[i])), ydataframe[i]) for i in range(len(xdataframe))]
    l=[]
    print(len(xdataframe))
    for i in range(len(xdataframe)):
        print(i, end = ' ')
        l.append((word_feats(WordPunctTokenizer().tokenize(xdataframe[i])), ydataframe[i]))
    return l
    
train_set = extract_feats(x_train, y_train)

4140


0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

 2200 2201 2202 2203 2204 2205 2206 2207 2208 2209 2210 2211 2212 2213 2214 2215 2216 2217 2218 2219 2220 2221 2222 2223 2224 2225 2226 2227 2228 2229 2230 2231 2232 2233 2234 2235 2236 2237 2238 2239 2240 2241 2242 2243 2244 2245 2246 2247 2248 2249 2250 2251 2252 2253 2254 2255 2256 2257 2258 2259 2260 2261 2262 2263 2264 2265 2266 2267 2268 2269 2270 2271 2272 2273 2274 2275 2276 2277 2278 2279 2280 2281 2282 2283 2284 2285 2286 2287 2288 2289 2290 2291 2292 2293 2294 2295 2296 2297 2298 2299 2300 2301 2302 2303 2304 2305 2306 2307 2308 2309 2310 2311 2312 2313 2314 2315 2316 2317 2318 2319 2320 2321 2322 2323 2324 2325 2326 2327 2328 2329 2330 2331 2332 2333 2334 2335 2336 2337 2338 2339 2340 2341 2342 2343 2344 2345 2346 2347 2348 2349 2350 2351 2352 2353 2354 2355 2356 2357 2358 2359 2360 2361 2362 2363 2364 2365 2366 2367 2368 2369 2370 2371 2372 2373 2374 2375 2376 2377 2378 2379 2380 2381 2382 2383 2384 2385 2386 2387 2388 2389 2390 2391 2392 2393 2394 2395 2396 2397 2398 2399

In [185]:
#Maxent train

algorithm = nltk.classify.MaxentClassifier.ALGORITHMS[0]
classifier = nltk.MaxentClassifier.train(train_set, algorithm, max_iter = 25)
classifier.show_most_informative_features(10)

  ==> Training (25 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -1.09861        0.333
             2          -1.09783        0.869
             3          -1.09705        0.869
             4          -1.09626        0.869
             5          -1.09548        0.869
             6          -1.09470        0.869
             7          -1.09392        0.869
             8          -1.09314        0.869
             9          -1.09237        0.869
            10          -1.09159        0.869
            11          -1.09081        0.869
            12          -1.09004        0.869
            13          -1.08926        0.869
            14          -1.08849        0.869
            15          -1.08772        0.869
            16          -1.08695        0.870
            17          -1.08617        0.870
            18          -1.08540        0.870
            19          -1.08463        0.870
  

In [188]:
#Maxent testing on all banks and performance index calculation from classifier predictions

values = {
    "compliment": 1,
    "miscellaneous": 0.5,
    "displeasure": 0
}

results = {}

direc = os.fsencode('./test')
for File in os.listdir(direc):
    filename = os.fsdecode(File)
    print(filename)
    try:
        test = pd.read_csv("./test/" + filename, header = None)
        for i in range(len(test[0])):
            test[0][i] = processTweet(test[0][i])
            
        x_test = clean_tweets(test)
        y_test = test[1]

        test_set = extract_feats(x_test, y_test)
        y_true = y_test
        y_pred = [classifier.classify(i[0]) for i in test_set]
        
        print()
        
        sum = 0
        for i in y_pred:
            sum += values[i]
        result = {}
        result['performance_index'] = sum * 100 / len(y_pred)
        result['classification_report'] = classification_report(y_true, y_pred)
        result['tweets'] = len(y_pred)
        results[filename[:-4]] = result
    except:
        pass

for i in sorted(results.items(), key = lambda x: ( -(getitem(x[1],'tweets')), -(getitem(x[1],'performance_index')) )):
    print((i[0] + ": ").rjust(40) + ("{:.2f}".format(i[1]['performance_index'])).rjust(6) + "% in " + str(i[1]['tweets']).rjust(4) + " tweets")
    print(i[1]['classification_report'])

Allahabad Bank.csv
2
0 1 
Andhra Bank.csv
Axis Bank.csv
500
0 Bank of Baroda.csv
138
0 1 Bank of India.csv
6
0 1 2 3 4 Bank of Maharashtra.csv
1
0 
Banking Ombudsman.csv
2
0 Canara Bank.csv
3
0 1 2 Central Bank.csv
2
0 1 
Citibank.csv
11
0 Corporation Bank.csv
3
0 1 2 
DBS Bank.csv
DCB Bank.csv
1
0 DNS Bank.csv
Federal Bank.csv
2
0 1 
Financial Ombudsman.csv
HDFC Bank.csv
500
0 1 HSBC.csv
2
0 ICICI Bank.csv
500
0 1 2 IDBI Bank.csv
11
0 1 2 IDFC First Bank.csv
1
0 
Indian Overseas Bank.csv
1
0 
IndusInd Bank.csv
13
0 1 Karnataka Bank.csv
Kotak Bank.csv
500
0 1 2 3 MobiKwik.csv
1
0 
Nainital Bank.csv
National Payments Corporation of India.csv
2
0 Oriental Bank.csv
2
0 PayTM.csv
5
0 1 2 3 4 
Post Office.csv
3
0 Punjab National Bank.csv
19
0 1 2 3 4 5 Reliance.csv
Reserve Bank of India.csv
253
0 1 2 3 Saraswat Bank.csv
State Bank of India.csv
500
0 1 State Bank of Mysore.csv
SVC Bank.csv
Syndicate Bank.csv
Tata Capital.csv
UCO Bank.csv
2
0 1 Union Bank of India.csv
3
0 1 2 
Vijaya Bank.csv

In [189]:
print(len(results))

12
